In [1]:
import pandas as pd

# Carregar o arquivo Excel
try:
    df = pd.read_excel('Hemoprod_CE.xlsx')
    
    # Nome da coluna a ser analisada
    coluna = 'Razão Social - Nome Fantasia '
    
    # Verificar se a coluna existe no DataFrame
    if coluna in df.columns:
        # Obter os valores distintos e a contagem
        valores_distintos = df[coluna].dropna().unique()
        contagem = len(valores_distintos)
        
        print(f'A coluna "{coluna}" tem {contagem} valores distintos.')
        print('Valores:')
        for valor in valores_distintos:
            print(f'- {valor}')
    else:
        print(f'A coluna "{coluna}" não foi encontrada no arquivo.')
        print('As colunas disponíveis são:')
        for c in df.columns:
            print(f'- {c}')
except FileNotFoundError:
    print('O arquivo Hemoprod_CE.xlsx não foi encontrado.')
except Exception as e:
    print(f'Ocorreu um erro: {e}')

O arquivo Hemoprod_CE.xlsx não foi encontrado.


In [2]:
# Supondo que 'df' é seu DataFrame

# 1. Defina o índice da coluna que deseja renomear
# A coluna problemática está na posição 10 (a 11ª)
indice_da_coluna = 10

# 2. Defina o novo nome
novo_nome = 'tipo_informacao'

# 3. Crie uma lista mutável com os nomes das colunas
lista_de_colunas = df.columns.tolist()

# 4. Substitua o nome da coluna no índice especificado
lista_de_colunas[indice_da_coluna] = novo_nome

# 5. Atribua a lista de volta ao df.columns
df.columns = lista_de_colunas

# Confirmação
print(f"Coluna na posição {indice_da_coluna} foi renomeada para '{novo_nome}'.")

# Imprime as primeiras colunas para confirmar (o novo nome estará na 11ª posição)
print("\nNovas colunas:")
print(df.columns.tolist()[:15])

NameError: name 'df' is not defined

In [ ]:
nomes_das_colunas = df.columns

primeiras_20_colunas = nomes_das_colunas[:20]

print("Nomes das 20 primeiras colunas:")
print(primeiras_20_colunas)

Nomes das 20 primeiras colunas:
Index(['ID da resposta', 'Data de envio', 'Última página', 'Idioma inicial',
       'Semente', 'Código de acesso', 'Data de início', 'Data da última ação',
       'Endereço IP', 'IDENTIFICAÇÃO DO DADO ', 'tipo_informacao',
       'Ano de referência  ', 'Período de referência',
       'IDENTIFICAÇÃO DO ESTABELECIMENTO ', 'Município',
       'Razão Social - Nome Fantasia ',
       'Razão Social - Nome Fantasia  [Outros]', 'CNPJ',
       'Tipo de estabelecimento', 'Natureza do estabelecimento'],
      dtype='object')


In [ ]:
# Isso retornará uma Series onde o índice são os valores e os dados são suas contagens
contagem_valores = df['tipo_informacao'].value_counts()

print("\nContagem de cada valor distinto em 'nome_da_coluna':")
print(contagem_valores)


Contagem de cada valor distinto em 'nome_da_coluna':
tipo_informacao
ORIGINAL        51
RETIFICADORA    16
Name: count, dtype: int64


In [ ]:
# 1. Converta a coluna de envio para o formato datetime
# Se já estiver em datetime, esta linha não fará mal.
df['Data de envio'] = pd.to_datetime(df['Data de envio'])

# 2. Renomeie as colunas de identificação para facilitar (opcional, mas recomendado)
# Lembre-se de usar os nomes exatos das colunas.
# Assumindo que você renomeou 'Ano de referência ' e 'Período de referência'
# e que 'CNPJ' já é um bom nome.
df.rename(columns={
    'Ano de referência\xa0 ': 'ano_referencia',
    'Período de referência': 'mes_referencia'
}, inplace=True)

In [ ]:
# 1. Definir as colunas de identificação (a "chave" para o envio)
chaves_identificacao = ['ano_referencia', 'mes_referencia', 'CNPJ']

# 2. Ordenar o DataFrame pela data de envio
# Isso garante que a última ocorrência (keep='last') seja a mais recente.
df_ordenado = df.sort_values(by='Data de envio', ascending=True)

# 3. Remover duplicatas, mantendo a última ocorrência (o último envio)
df_ultimo_envio = df_ordenado.drop_duplicates(
    subset=chaves_identificacao, # As colunas que definem um envio único
    keep='last'                   # Manter o último (mais recente)
)

print(f"Total de envios originais: {len(df)}")
print(f"Total de últimos envios únicos: {len(df_ultimo_envio)}")

# Agora você deve trabalhar com o DataFrame 'df_ultimo_envio'

Total de envios originais: 228
Total de últimos envios únicos: 65


In [ ]:
import pandas as pd

# Carregar os arquivos
hemoprod = pd.read_excel('hemoprod_ultimos_envios.xlsx')
dicionario = pd.read_excel('dicionario_colunas.xlsx')

# Criar um dicionário de mapeamento: nome_original -> nome_sql
# Remover espaços extras das colunas do dicionário
dicionario['nome_original'] = dicionario['nome_original'].str.strip()
dicionario['nome_sql'] = dicionario['nome_sql'].str.strip()

# Criar o mapeamento
mapeamento = dict(zip(dicionario['nome_original'], dicionario['nome_sql']))

# Também remover espaços extras das colunas do hemoprod
hemoprod.columns = hemoprod.columns.str.strip()

# Renomear as colunas do hemoprod usando o dicionário
# Apenas renomeia as colunas que existem no dicionário
hemoprod_renomeado = hemoprod.rename(columns=mapeamento)

# Verificar quais colunas foram renomeadas
colunas_renomeadas = [col for col in hemoprod.columns if col in mapeamento]
colunas_nao_encontradas = [col for col in hemoprod.columns if col not in mapeamento]

print(f"Total de colunas no hemoprod: {len(hemoprod.columns)}")
print(f"Colunas renomeadas: {len(colunas_renomeadas)}")
print(f"Colunas não encontradas no dicionário: {len(colunas_nao_encontradas)}")

if colunas_nao_encontradas:
    print("\nColunas que não foram encontradas no dicionário:")
    for col in colunas_nao_encontradas[:10]:  # Mostrar apenas as primeiras 10
        print(f"  - {col}")

# Salvar o arquivo com os nomes novos
hemoprod_renomeado.to_excel('hemoprod_ultimos_envios_renomeado.xlsx', index=False)

print("\n✅ Arquivo 'hemoprod_ultimos_envios_renomeado.xlsx' gerado com sucesso!")

Total de colunas no hemoprod: 269
Colunas renomeadas: 61
Colunas não encontradas no dicionário: 208

Colunas que não foram encontradas no dicionário:
  - Tipo de Informação  Antes de responder ao formulário, declare o tipo de informação que será inserida.
  - Razão Social - Nome Fantasia  [Outros]
  - Cite os estabelecimentos que compõem a rede  Informe o Tipo de Estabelecimento, o Nome Fantasia e o Município de localização de cada um.
  - CNES - Cadastro Nacional de Estabelecimentos de Saúde
  - 2. Triagem Clínica  2.1 Total de candidatos quanto ao tipo de doação  [Espontânea][Aptos]
  - 2. Triagem Clínica  2.1 Total de candidatos quanto ao tipo de doação  [Espontânea][Inaptos]
  - 2. Triagem Clínica  2.1 Total de candidatos quanto ao tipo de doação  [Reposição][Aptos]
  - 2. Triagem Clínica  2.1 Total de candidatos quanto ao tipo de doação  [Reposição][Inaptos]
  - 2. Triagem Clínica  2.1 Total de candidatos quanto ao tipo de doação  [Autóloga][Aptos]
  - 2. Triagem Clínica  2.1 Tota

In [ ]:
import pandas as pd

hemoprod = pd.read_excel('hemoprod_ultimos_envios.xlsx')
dic = pd.read_excel('dicionario_colunas.xlsx')

# Limpar espaços
dic['nome_sql'] = dic['nome_sql'].astype(str).str.strip()

# Limite pelo tamanho dos dados
n_cols = len(hemoprod.columns)
n_dic = len(dic)

# Criar nova lista de nomes, usando nome_sql quando existir, senão mantém nome atual
new_cols = []
for i, col in enumerate(hemoprod.columns):
    if i < n_dic and pd.notna(dic.loc[i, 'nome_sql']) and str(dic.loc[i, 'nome_sql']).strip() != '':
        new_cols.append(dic.loc[i, 'nome_sql'])
    else:
        new_cols.append(col)

hemoprod.columns = new_cols
hemoprod.to_excel('hemoprod_ultimos_envios_renomeado_por_indice.xlsx', index=False)

print("✅ Gerado: hemoprod_ultimos_envios_renomeado_por_indice.xlsx")
print(f"Total colunas hemoprod: {n_cols} | Linhas dicionário: {n_dic}")

✅ Gerado: hemoprod_ultimos_envios_renomeado_por_indice.xlsx
Total colunas hemoprod: 269 | Linhas dicionário: 269


In [ ]:
import pandas as pd

# Supondo que 'df' é seu DataFrame

# 1. Altera a opção para exibir no máximo 500 colunas
# (Você pode ajustar o número se tiver mais colunas)
pd.set_option('display.max_columns', 500)

# 2. Exibe as primeiras 5 linhas com todas as colunas
hemoprod.head()

,id,data_envio,ultima_pagina,idioma_inicial,semente,codigo_acesso,data_inicio,data_ultima_acao,ip,identificacao_dado,tipo_envio,ano_referencia,periodo_referencia,identificacao_estabelecimento,municipio,razao_social_nome_fantasia,razao_social_nome_fantasia_outros,cnpj,tipo_estabelecimento,natureza_estabelecimento,dados_informados_referem_se,rede_estabelecimento,cnes,endereco,triagem_clinica_total_doacao_espontanea_aptos,triagem_clinica_total_doacao_espontanea_inaptos,triagem_clinica_total_doacao_reposicao_aptos,triagem_clinica_total_doacao_reposicao_inaptos,triagem_clinica_total_doacao_autologa_aptos,triagem_clinica_total_doacao_autologa_inaptos,total_doador_primeira_vez_aptos,total_doador_primeira_vez_inaptos,total_doador_repeticao_aptos,total_doador_repeticao_inaptos,total_doador_esporadico_aptos,total_doador_esporadico_inaptos,total_doador_masculino_aptos,total_doador_masculino_inaptos,total_doador_feminino_aptos,total_doador_feminino_inaptos,total_doador_menor_de_18_anos_aptos,total_doador_menor_de_18_anos_inaptos,total_doador_18_ate_29_anos_aptos,total_doador_18_ate_29_anos_inaptos,total_doador_acima_de_29_anos_aptos,total_doador_acima_de_29_anos_inaptos,total_candidatos_inaptos_anemia_masculino,total_candidatos_inaptos_anemia_feminino,total_candidatos_inaptos_anemia_total,total_candidatos_inaptos_hipertensao_masculino,total_candidatos_inaptos_hipertensao_feminino,total_candidatos_inaptos_hipertensao_total,total_candidatos_inaptos_hipotensao_masculino,total_candidatos_inaptos_hipotensao_feminino,total_candidatos_inaptos_hipotensao_total,total_candidatos_inaptos_alcoolismo_masculino,total_candidatos_inaptos_alcoolismo_feminino,total_candidatos_inaptos_alcoolismo_total,total_candidatos_inaptos_comportamento_risco_dst_masculino,total_candidatos_inaptos_comportamento_risco_dst_feminino,total_candidatos_inaptos_comportamento_risco_dst_total,total_candidatos_inaptos_uso_drogas_masculino,total_candidatos_inaptos_uso_drogas_feminino,total_candidatos_inaptos_uso_drogas_total,total_candidatos_inaptos_hepatite_masculino,total_candidatos_inaptos_hepatite_feminino,total_candidatos_inaptos_hepatite_total,total_candidatos_inaptos_doenca_chagas_masculino,total_candidatos_inaptos_doenca_chagas_feminino,total_candidatos_inaptos_doenca_chagas_total,total_candidatos_inaptos_malaria_masculino,total_candidatos_inaptos_malaria_feminino,total_candidatos_inaptos_malaria_total,total_candidatos_inaptos_outras_masculino,total_candidatos_inaptos_outras_feminino,total_candidatos_inaptos_outras_total,coleta_total_candidatos_desistentes,total_interrupcoes_coleta_dificuldade_puncao_venosa,total_interrupcoes_coleta_reacao_vagal,total_interrupcoes_coleta_outros_motivos,total_coletas_sangue_total,total_coletas_aferese,hemoprod_1_observacoes,exames_triagem_doenca_doenca_chagas_amostras_testadas,exames_triagem_doenca_doenca_chagas_amostras_reagentes,exames_triagem_doenca_hiv_amostras_testadas,exames_triagem_doenca_hiv_amostras_reagentes,exames_triagem_doenca_sifilis_amostras_testadas,`exames_triagem_doenca_sifilis_amostras_reagentes,exames_triagem_doenca_hepatite_b_hbs_ag_amostras_testadas,exames_triagem_doenca_hepatite_b_hbs_ag_amostras_reagentes,exames_triagem_doenca_hepatite_b_anti_hbc_amostras_testadas,exames_triagem_doenca_hepatite_b_anti_hbc_amostras_reagentes,exames_triagem_doenca_hepatite_c_amostras_testadas,exames_triagem_doenca_hepatite_c_amostras_reagentes,exames_triagem_doenca_htlv_i_ii_amostras_testadas,exames_triagem_doenca_htlv_i_ii_amostras_reagentes,exames_triagem_doenca_malaria_amostras_testadas,exames_triagem_doenca_malaria_amostras_reagentes,exames_triagem_doenca_hbv_teste_nat_amostras_testadas,exames_triagem_doenca_hbv_teste_nat_amostras_reagentes,exames_triagem_doenca_hcv_teste_nat_amostras_testadas,exames_triagem_doenca_hcv_teste_nat_amostras_reagentes,exames_triagem_doenca_hiv_teste_nat_amostras_testadas,exames_triagem_doenca_hiv_teste_nat_amostras_reagentes,imunohematologia_a_positivo_doador,imunohematologia_a_positivo_receptor,imunohe

In [ ]:
hemoprod.describe()

,id,data_envio,ultima_pagina,semente,data_inicio,data_ultima_acao,identificacao_dado,ano_referencia,identificacao_estabelecimento,razao_social_nome_fantasia_outros,cnpj,cnes,endereco,triagem_clinica_total_doacao_espontanea_aptos,triagem_clinica_total_doacao_espontanea_inaptos,triagem_clinica_total_doacao_reposicao_aptos,triagem_clinica_total_doacao_reposicao_inaptos,triagem_clinica_total_doacao_autologa_aptos,triagem_clinica_total_doacao_autologa_inaptos,total_doador_primeira_vez_aptos,total_doador_primeira_vez_inaptos,total_doador_repeticao_aptos,total_doador_repeticao_inaptos,total_doador_esporadico_aptos,total_doador_esporadico_inaptos,total_doador_masculino_aptos,total_doador_masculino_inaptos,total_doador_feminino_aptos,total_doador_feminino_inaptos,total_doador_menor_de_18_anos_aptos,total_doador_menor_de_18_anos_inaptos,total_doador_18_ate_29_anos_aptos,total_doador_18_ate_29_anos_inaptos,total_doador_acima_de_29_anos_aptos,total_doador_acima_de_29_anos_inaptos,total_candidatos_inaptos_anemia_masculino,total_candidatos_inaptos_anemia_feminino,total_candidatos_inaptos_anemia_total,total_candidatos_inaptos_hipertensao_masculino,total_candidatos_inaptos_hipertensao_feminino,total_candidatos_inaptos_hipertensao_total,total_candidatos_inaptos_hipotensao_masculino,total_candidatos_inaptos_hipotensao_feminino,total_candidatos_inaptos_hipotensao_total,total_candidatos_inaptos_alcoolismo_masculino,total_candidatos_inaptos_alcoolismo_feminino,total_candidatos_inaptos_alcoolismo_total,total_candidatos_inaptos_comportamento_risco_dst_masculino,total_candidatos_inaptos_comportamento_risco_dst_feminino,total_candidatos_inaptos_comportamento_risco_dst_total,total_candidatos_inaptos_uso_drogas_masculino,total_candidatos_inaptos_uso_drogas_feminino,total_candidatos_inaptos_uso_drogas_total,total_candidatos_inaptos_hepatite_masculino,total_candidatos_inaptos_hepatite_feminino,total_candidatos_inaptos_hepatite_total,total_candidatos_inaptos_doenca_chagas_masculino,total_candidatos_inaptos_doenca_chagas_feminino,total_candidatos_inaptos_doenca_chagas_total,total_candidatos_inaptos_malaria_masculino,total_candidatos_inaptos_malaria_feminino,total_candidatos_inaptos_malaria_total,total_candidatos_inaptos_outras_masculino,total_candidatos_inaptos_outras_feminino,total_candidatos_inaptos_outras_total,coleta_total_candidatos_desistentes,total_interrupcoes_coleta_dificuldade_puncao_venosa,total_interrupcoes_coleta_reacao_vagal,total_interrupcoes_coleta_outros_motivos,total_coletas_sangue_total,total_coletas_aferese,hemoprod_1_observacoes,exames_triagem_doenca_doenca_chagas_amostras_testadas,exames_triagem_doenca_doenca_chagas_amostras_reagentes,exames_triagem_doenca_hiv_amostras_testadas,exames_triagem_doenca_hiv_amostras_reagentes,exames_triagem_doenca_sifilis_amostras_testadas,`exames_triagem_doenca_sifilis_amostras_reagentes,exames_triagem_doenca_hepatite_b_hbs_ag_amostras_testadas,exames_triagem_doenca_hepatite_b_hbs_ag_amostras_reagentes,exames_triagem_doenca_hepatite_b_anti_hbc_amostras_testadas,exames_triagem_doenca_hepatite_b_anti_hbc_amostras_reagentes,exames_triagem_doenca_hepatite_c_amostras_testadas,exames_triagem_doenca_hepatite_c_amostras_reagentes,exames_triagem_doenca_htlv_i_ii_amostras_testadas,exames_triagem_doenca_htlv_i_ii_amostras_reagentes,exames_triagem_doenca_malaria_amostras_testadas,exames_triagem_doenca_malaria_amostras_reagentes,exames_triagem_doenca_hbv_teste_nat_amostras_testadas,exames_triagem_doenca_hbv_teste_nat_amostras_reagentes,exames_triagem_doenca_hcv_teste_nat_amostras_testadas,exames_triagem_doenca_hcv_teste_nat_amostras_reagentes,exames_triagem_doenca_hiv_teste_nat_amostras_testadas,exames_triagem_doenca_hiv_teste_nat_amostras_reagentes,imunohematologia_a_positivo_doador,imunohematologia_a_positivo_receptor,imunohematologia_b_positivo_doador,imunohematologia_b_positivo_receptor,imunohematologia_ab_positivo_doador,imunohematologia_ab_positivo_receptor,imunohematologia_o_positivo_doador,imunohematologia_o_p

## Hemo prod Alagoas

Index(['id', 'data_envio', 'ultima_pagina', 'idioma_inicial', 'semente',
       'codigo_acesso', 'data_inicio', 'data_ultima_acao', 'ip',
       'identificacao_dado',
       ...
       'reacoes_transfusionais_reacao_febril_nao_hemolitica',
       'reacoes_transfusionais_reacao_hemolitica',
       'reacoes_transfusionais_reacao_alergica',
       'reacoes_transfusionais_choque_bacteriano',
       'reacoes_transfusionais_alteracoes_metabolicas',
       'reacoes_transfusionais_sobrecarga_volemica',
       'reacoes_transfusionais_outras_reacoes',
       'envio_plasma_producao_hemoderivados_plasma_fresco_congelado',
       'envio_plasma_producao_hemoderivados_plasma_comum',
       'hemoprod_3_observacoes'],
      dtype='object', length=269)

: 